- Setup 50k/183k rows for portfolio purpose .

- All the comments calculation from real data 183k rows 

In [1]:
import pandas as pd

train = pd.read_csv('/kaggle/input/playground-series-s4e9/train.csv', nrows=50000)
test = pd.read_csv('/kaggle/input/playground-series-s4e9/test.csv', nrows=50000)

In [2]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            50000 non-null  int64 
 1   brand         50000 non-null  object
 2   model         50000 non-null  object
 3   model_year    50000 non-null  int64 
 4   milage        50000 non-null  int64 
 5   fuel_type     48661 non-null  object
 6   engine        50000 non-null  object
 7   transmission  50000 non-null  object
 8   ext_col       50000 non-null  object
 9   int_col       50000 non-null  object
 10  accident      49362 non-null  object
 11  clean_title   44406 non-null  object
 12  price         50000 non-null  int64 
dtypes: int64(4), object(9)
memory usage: 5.0+ MB


In [3]:
train_df = train.fillna({col: 0 for col in train.select_dtypes(include=['number']).columns})
train_df = train.fillna({col: 'unknown' for col in train.select_dtypes(include=['object']).columns})

In [4]:
test_df = test.fillna({col: 0 for col in test.select_dtypes(include=['number']).columns})
test_df = test.fillna({col: 'unknown' for col in test.select_dtypes(include=['object']).columns})

In [5]:
# Define a function to cap outliers based on the 1st and 99th percentiles
def cap_outliers(df, column):
    lower_limit = df[column].quantile(0.01)
    upper_limit = df[column].quantile(0.99)
    df[column] = df[column].clip(lower=lower_limit, upper=upper_limit)

# Apply the function to numeric columns in train_df
numeric_columns = train_df.select_dtypes(include=['number']).columns
for col in numeric_columns:
    cap_outliers(train_df, col)

# Apply the function to numeric columns in test_df
numeric_columns_test = test_df.select_dtypes(include=['number']).columns
for col in numeric_columns_test:
    cap_outliers(test_df, col)

In [6]:
# Filter the dataset for the most recent year (2022)
recent_year = 2022
recent_purchases = train_df[train_df['model_year'] == recent_year]

# Calculate the percentage of customers who made purchases in the recent year
percentage_recent_purchases = (len(recent_purchases) / len(train_df)) * 100
percentage_recent_purchases

8.236

- 8.35% of customers made purchases in the most recent year (2022)

In [7]:
# Find the range of years in the 'model_year' column
unique_years = train_df['model_year'].unique()
unique_years.sort()
unique_years

array([2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010,
       2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021,
       2022, 2023])

In [8]:
# Calculate the percentage of customers who made purchases for each year
purchase_percentage_by_year = train_df['model_year'].value_counts(normalize=True) * 100

# Find the year with the most and least percentage of purchases
most_purchases_year = purchase_percentage_by_year.idxmax()
most_purchases_percentage = purchase_percentage_by_year.max()

least_purchases_year = purchase_percentage_by_year.idxmin()
least_purchases_percentage = purchase_percentage_by_year.min()

most_purchases_year, most_purchases_percentage, least_purchases_year, least_purchases_percentage

(2021, 9.712, 2002, 0.688)

- The year with the most percentage of customers who made purchases is 2021, with approximately 9.65%.

-  The year with the least percentage of customers who made purchases is 2002, with approximately 0.68%.

In [9]:
# Calculate the average amount spent per transaction
average_transaction_amount = train_df['price'].mean()
average_transaction_amount

41454.99614

- The average amount spent per transaction is 41,182.28. 

In [10]:
# Calculate total sales for each year
total_sales_by_year = train_df.groupby('model_year')['price'].sum()

# Calculate the percentage increase in sales over the years
percentage_increase_sales = total_sales_by_year.pct_change() * 100

percentage_increase_sales

model_year
2000          NaN
2001   -38.337897
2002     5.600326
2003    25.659492
2004    40.611584
2005    41.016325
2006   -14.561103
2007    73.738219
2008     5.799909
2009   -37.319976
2010    33.112733
2011    51.374178
2012     7.083522
2013    32.871973
2014    23.846683
2015    72.006036
2016    28.724243
2017     4.880967
2018    59.690636
2019     0.500131
2020    23.149907
2021    24.539000
2022    -7.367987
2023   -25.860990
Name: price, dtype: float64

- The percentage increase in sales varies by year.

- For example, there was a 76.68% increase in sales in 2007 compared to 2006, while there was a -25.95% decrease in 2023 compared to 2022

In [11]:
completed_action = train_df[train_df['price'] > 0]
engaged_customers = train_df.drop_duplicates(subset=['id'])

# Calculate the percentage of visitors who make a purchase
percentage_completed_action = (len(completed_action) / len(engaged_customers)) * 100
percentage_completed_action

102.03665156524224

- 102.04% of visitors make a purchase.

In [12]:
# Calculate the percentage of customers who stop purchasing over a defined period
percentage_stopped_purchasing = ((len(engaged_customers) - len(completed_action)) / len(engaged_customers)) * 100
percentage_stopped_purchasing

-2.0366515652422352

- -2.04% of customers stopped purchasing over the defined period.

In [13]:
# Define price ranges for mapping
cheap_threshold = average_transaction_amount * 0.5
affordable_threshold = average_transaction_amount * 1.5

# Map prices into categories
def map_price_category(price):
    if price <= cheap_threshold:
        return 'cheap'
    elif price <= affordable_threshold:
        return 'affordable'
    else:
        return 'expensive'

train_df['price_category'] = train_df['price'].apply(map_price_category)

In [14]:
test_df['price_category'] = 'unknown'

In [15]:
# Map 'accident' column into False and True
def map_accident(value):
    if value == 'None reported':
        return False
    else:
        return True

train_df['accident_category'] = train_df['accident'].apply(map_accident)
train_df['accident_category']

0        False
1         True
2        False
3        False
4        False
         ...  
49995     True
49996     True
49997     True
49998    False
49999    False
Name: accident_category, Length: 50000, dtype: bool

In [16]:
# Map 'accident' column into False and True
def map_accident(value):
    if value == 'None reported':
        return False
    else:
        return True

test_df['accident_category'] = test_df['accident'].apply(map_accident)
test_df['accident_category']

0        False
1        False
2        False
3        False
4        False
         ...  
49995    False
49996    False
49997     True
49998     True
49999    False
Name: accident_category, Length: 50000, dtype: bool

In [17]:
train_df_encoded = pd.get_dummies(train_df, columns=['brand', 'model', 'model_year', 'milage', 'fuel_type', 'engine', 'transmission', 'ext_col', 'int_col', 'accident', 'clean_title', 'price_category', 'accident_category'])
test_df_encoded = pd.get_dummies(test_df, columns=['brand', 'model', 'model_year', 'milage', 'fuel_type', 'engine', 'transmission', 'ext_col', 'int_col', 'accident', 'clean_title', 'price_category', 'accident_category'])

In [18]:
# Align columns between train and test datasets
train_df_encoded, test_df_encoded = train_df_encoded.align(test_df_encoded, join='left', axis=1)
test_df_encoded.fillna(0, inplace=True)

In [19]:
X = ['brand', 'model', 'model_year', 'milage', 'fuel_type', 'engine', 'transmission', 'ext_col', 'int_col', 'accident', 'clean_title', 'price_category', 'accident_category']
y = 'price'

In [20]:
X = train_df_encoded.drop(columns=['id', 'price'])
y = train_df_encoded['price']

In [21]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25, random_state=42)

In [22]:
%%time

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
import numpy as np

# Initialize KFold
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Initialize the scaler
scaler = StandardScaler()

# Initialize the model
model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)

rmse_scores = []

for train_index, val_index in kf.split(X_train):
    # Split the data into training and validation sets
    X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]

    # Scale the features
    X_train_fold_scaled = scaler.fit_transform(X_train_fold)
    X_val_fold_scaled = scaler.transform(X_val_fold)

    # Train the model
    model.fit(X_train_fold_scaled, y_train_fold)

    # Make predictions on the validation set
    y_val_pred = model.predict(X_val_fold_scaled)

    # Evaluate RMSE
    rmse = mean_squared_error(y_val_fold, y_val_pred, squared=False)
    rmse_scores.append(rmse)

# Calculate the average RMSE
average_rmse = np.mean(rmse_scores)
average_rmse

CPU times: user 8min 23s, sys: 10.7 s, total: 8min 34s
Wall time: 2min 30s


23599.94587829872

In [23]:
import joblib

# Save the trained pipeline to a .pkl file
joblib.dump(model, 'used_car_model.pkl')

['used_car_model.pkl']

In [24]:
X_test = test_df_encoded.drop(columns=['id', 'price'])

In [25]:
X_test_scaled = scaler.fit_transform(X_test)

In [26]:
# Make predictions on test set
y_test_pred = model.predict(X_test_scaled)

In [27]:
predicted_prices = y_test_pred

In [29]:
predicted_prices

array([15283.023, 13194.121, 12863.843, ..., 13376.152, 14787.412,
       13194.121], dtype=float32)